# Repo

In [ ]:
#@title Clone repo
from google.colab import userdata
github_token = userdata.get('github_token')

%cd /content
!git clone https://$github_token@github.com/andyrightnow/fish-speech.git

In [ ]:
#@title Checkout and pull code

%cd /content/fish-speech
!git checkout andy
!git pull

!git config --global user.email "naruto199507@gmail.com"
!git config --global user.name "AndyRightNow"

In [ ]:
#@title Install `uv`

!curl -LsSf https://astral.sh/uv/install.sh | sh
!source $HOME/.local/bin/env

In [ ]:
#@title Install dependencies

!uv pip install --system torch==2.4.1 torchvision==0.19.1 torchaudio==2.4.1 m4b-util==0.4.1


!apt install libsox-dev ffmpeg

!apt install build-essential \
    cmake \
    libasound-dev \
    portaudio19-dev \
    libportaudio2 \
    libportaudiocpp0

!uv pip install --system -e .[stable]

from google.colab import output

output.clear()

In [ ]:
#@title Download checkpoints

from google.colab import userdata
huggingface_token = userdata.get('huggingfaceToken')

!uv pip install --system -U "huggingface_hub[cli]"
!huggingface-cli login --token $huggingface_token  --add-to-git-credential
!huggingface-cli download fishaudio/fish-speech-1.5 --local-dir checkpoints/fish-speech-1.5

In [ ]:
#@title Upload input files

%cd /content/fish-speech/generate_pipelines
!mkdir input
%cd input

from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

%cd /content/fish-speech

# Run generate script

In [ ]:
#@title __Archived scripts

# !python generate_pipelines/main.py --prompt-name tifa --input-name we-after-everything --insert-segment-title true --segment-title-language en --max-sem-input-count 2500
# !python generate_pipelines/convert_mp3.py --input-name we-after-everything --prompt-name tifa --max-sem-input-count 2500
# !python generate_pipelines/convert_mp3.py --input-name hengxingshike --prompt-name weisan

# ------------------------------------ Archived ------------------------------------


In [ ]:
# @title Main scripts
# Run scripts here

from google.colab import output

output.clear()

!LOGURU_LEVEL=SUCCESS python generate_pipelines/main.py --prompt-name xianyun --input-name wanjiaqingjiuwei --generate-mp3 true --max-sem-input-count 999

In [ ]:
#@title Bind audiobook


hash = '' # @param {type: "string"}
title = 'Wanjiaqingjiuwei Part 2' # @param {type: "string"}
author = '时微月上' # @param {type: "string"}

if hash and title and author:
  !python -m m4b_util bind generate_pipelines/output/mp3/{hash} -o generate_pipelines/output -t "{title}" --use-filename -a "{author}"

In [ ]:
#@title Commit and push states

import os

%cd /content/fish-speech
# Check for modified files in generate_pipelines/states
!git status -s generate_pipelines/states

# Check if there's any output from the git status command
if os.system('git status -s generate_pipelines/states | grep "states"') != 0:
  print("No changes detected in generate_pipelines/states")
else:
  confirm = input("Are you sure?(y/n)")

  if confirm == "y":
    !git add generate_pipelines/states
    !git commit -m "Commit states"
    !git push

In [ ]:
#@title (Dangerous) Hard reset to origin

!git checkout andy

confirm = input("Are you sure?(y/n)")

if confirm == "y":
  !git fetch --all && git reset --hard origin/andy

# Debug

In [ ]:
debug = False  # @param {type: "boolean"}

In [ ]:
if debug:
  %cd /content/fish-speech
  !python generate_pipelines/debug_print_shelve.py --input-name hengxingshike --prompt-name weisan

  !cat generate_pipelines/states/states.json

# Generate voice prompts

In [ ]:
generate_voice_prompts = False  # @param {type: "boolean"}

In [ ]:
# prompt: if `generate_voice_prompts` is true, first prompt to ask for an input audio file and an audio text, and then save the text file to generate_pipelines/{audio_file_name}.txt
from google.colab import files
from pathlib import Path
import time

if generate_voice_prompts:
  %cd /content/fish-speech
  tmp_voice_prompts_dir_name = f"_tmp_voice_prompts_{int(time.time())}"
  !mkdir $tmp_voice_prompts_dir_name
  %cd $tmp_voice_prompts_dir_name

  uploaded = files.upload()

  for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))

  audio_text = input("Enter the text for the audio: ")

  %cd /content/fish-speech

  audio_file_name = next(iter(uploaded.keys()), None)
  audio_file_name_stem = Path(audio_file_name).stem

  !python tools/vqgan/inference.py -i "{tmp_voice_prompts_dir_name}/{audio_file_name}" --checkpoint-path "checkpoints/fish-speech-1.5/firefly-gan-vq-fsq-8x1024-21hz-generator.pth" -o "generate_pipelines/{audio_file_name_stem}.wav"

  !echo "{audio_text}" > "generate_pipelines/{audio_file_name_stem}.txt"

  !git status -s generate_pipelines
  confirm = input("Are you sure you want to commit the voice prompts?(y/n)")

  if confirm == 'y':
    !git add generate_pipelines/*.npy generate_pipelines/*.txt
    !git commit -m "Commit voice prompts"
    !git push